In [1]:
import random
from collections import Counter

class Carta:
    palos = ['Corazones', 'Diamantes', 'Tréboles', 'Picas']
    valores = ['2', '3', '4', '5', '6', '7', '8', '9', '10',
               'Jota', 'Reina', 'Rey', 'As']

    def __init__(self, valor, palo):
        if valor not in Carta.valores:
            raise ValueError(f"Valor inválido: {valor}")
        if palo not in Carta.palos:
            raise ValueError(f"Palo inválido: {palo}")
        self.valor = valor
        self.palo = palo

    def __str__(self):
        return f"{self.valor} de {self.palo}"

    def valor_numérico(self):
        """Devuelve el valor numérico de la carta para comparación."""
        if self.valor.isdigit():
            return int(self.valor)
        elif self.valor == 'As':
            return 14
        elif self.valor == 'Rey':
            return 13
        elif self.valor == 'Reina':
            return 12
        elif self.valor == 'Jota':
            return 11


In [2]:
class Baraja:
    def __init__(self):
        self.cartas = [Carta(valor, palo) for palo in Carta.palos for valor in Carta.valores]
        self.barajar()

    def barajar(self):
        random.shuffle(self.cartas)

    def repartir(self):
        if len(self.cartas) == 0:
            raise ValueError("Todas las cartas han sido repartidas")
        return self.cartas.pop()


In [3]:
class Mano:
    def __init__(self):
        self.cartas = []

    def agregar_carta(self, carta):
        if len(self.cartas) >= 5:
            raise ValueError("La mano ya tiene 5 cartas")
        self.cartas.append(carta)

    def reemplazar_carta(self, índice, nueva_carta):
        if índice < 0 or índice >= len(self.cartas):
            raise IndexError("Índice de carta inválido")
        self.cartas[índice] = nueva_carta

    def evaluar_mano(self):
        valores = [carta.valor for carta in self.cartas]
        palos = [carta.palo for carta in self.cartas]
        conteo_valores = Counter(valores)
        conteo_palos = Counter(palos)
        valores_numéricos = sorted([carta.valor_numérico() for carta in self.cartas], reverse=True)

        es_color = max(conteo_palos.values()) == 5
        es_escalera = False
        if valores_numéricos == list(range(valores_numéricos[0], valores_numéricos[0]-5, -1)):
            es_escalera = True
        # Caso especial: Escalera con As como 1 (As,2,3,4,5)
        if set(valores_numéricos) == set([14, 2, 3, 4, 5]):
            es_escalera = True
            valores_numéricos = [5, 4, 3, 2, 1]

        if es_escalera and es_color:
            return ("Escalera de Color", 50)
        elif 4 in conteo_valores.values():
            return ("Póker", 40)
        elif sorted(conteo_valores.values()) == [2, 3]:
            return ("Full", 30)
        elif es_color:
            return ("Color", 25)
        elif es_escalera:
            return ("Escalera", 20)
        elif 3 in conteo_valores.values():
            return ("Trío", 15)
        elif list(conteo_valores.values()).count(2) == 2:
            return ("Doble Pareja", 10)
        elif 2 in conteo_valores.values():
            return ("Pareja", 5)
        else:
            return ("Carta Alta", 1)

    def __str__(self):
        return ', '.join([f"{índice+1}: {carta}" for índice, carta in enumerate(self.cartas)])


In [4]:
class JuegoPoker:
    def __init__(self):
        self.baraja = Baraja()
        self.mano_jugador = Mano()
        self.puntos = 0

    def iniciar_juego(self):
        print("¡Bienvenido al Póker de 5 Cartas!")
        self.robar_mano_inicial()
        self.turno_jugador()
        self.evaluar_puntuación()

    def robar_mano_inicial(self):
        for _ in range(5):
            self.mano_jugador.agregar_carta(self.baraja.repartir())

    def turno_jugador(self):
        print("\nTu mano actual:")
        print(self.mano_jugador)
        elección = input("¿Deseas reemplazar alguna carta? (s/n): ").lower()
        if elección == 's':
            try:
                índices = input("Ingresa los números de las cartas a reemplazar (separados por espacio): ")
                índices = list(map(int, índices.strip().split()))
                if len(set(índices)) > 5:
                    print("Solo puedes reemplazar hasta 5 cartas.")
                    return
                for idx in índices:
                    if idx < 1 or idx > 5:
                        print(f"Número de carta inválido: {idx}")
                        continue
                    nueva_carta = self.baraja.repartir()
                    self.mano_jugador.reemplazar_carta(idx-1, nueva_carta)
                    print(f"Reemplazada carta {idx} por {nueva_carta}")
                print("\nTu nueva mano:")
                print(self.mano_jugador)
            except Exception as e:
                print(f"Error: {e}")
        else:
            print("No se reemplazaron cartas.")

    def evaluar_puntuación(self):
        evaluación, puntos_mano = self.mano_jugador.evaluar_mano()
        self.puntos += puntos_mano
        print("\n--- Resultados ---")
        print(f"Tu mano: {self.mano_jugador} -> {evaluación} ({puntos_mano} puntos)")
        print(f"Puntuación total: {self.puntos} puntos")


In [5]:
def main():
    juego = JuegoPoker()
    juego.iniciar_juego()

if __name__ == "__main__":
    main()


¡Bienvenido al Póker de 5 Cartas!

Tu mano actual:
1: 5 de Picas, 2: 3 de Diamantes, 3: 5 de Diamantes, 4: As de Tréboles, 5: 9 de Picas
¿Deseas reemplazar alguna carta? (s/n): s
Ingresa los números de las cartas a reemplazar (separados por espacio): 2 4 5
Reemplazada carta 2 por Jota de Diamantes
Reemplazada carta 4 por 6 de Corazones
Reemplazada carta 5 por 8 de Picas

Tu nueva mano:
1: 5 de Picas, 2: Jota de Diamantes, 3: 5 de Diamantes, 4: 6 de Corazones, 5: 8 de Picas

--- Resultados ---
Tu mano: 1: 5 de Picas, 2: Jota de Diamantes, 3: 5 de Diamantes, 4: 6 de Corazones, 5: 8 de Picas -> Pareja (5 puntos)
Puntuación total: 5 puntos
